In [ ]:
# Markov Attempt 4 
# All lyrics - way more lines made - same rhyme ranker as in attempt 1
# Changelog: 
# - Uploaded the fuction for updating the bad words. 
# - Uploaded a new function for recursively importing the cleaned lyrics. This way it's always updated. 
# - Changed the way which words are capitalized with a capitals.csv which can be changed easily
# - Made the imports faster by changing the folder hierarchy 
# - Added the ability to change main branch, not advised to use tho

In [ ]:
#@title Import Statements
!pip install PyGithub

# Package Imports
import random
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from urllib.request import urlopen # The default requests package
import requests # For making GitHub requests
from pprint import pprint # For pretty printing
from pathlib import Path # The Path class
import re  # For multiple delimiters seperating strings

# System Packages 
import base64
import os
import sys
import time

# Tensorflow 
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

# For the more advanced requests
sys.path.append("./PyGithub");
from github import Github
from getpass import getpass

In [ ]:
#@title Function Definitions 1
# Recursively Import the Data (AUTOMATIC)

def _decode_and_write(file__, path_):
    data = file__.decoded_content
    data = data.decode('utf-8')[1:]
    with open(path_, 'w') as writefile:
        writefile.write(data) 
    data = data.splitlines()
    data_rows = []
    for count, word in enumerate(data):
        if count>0:
            data_rows.append(word.split(','))
    data = pd.DataFrame(data_rows)
    data = data.to_numpy()
    return data


def import_github(path_name="AllLyrics.txt"):
    """
    Function for importing the github file
    path_name: str
    output: None
    """
    g = Github(getpass("Enter your PAT key ")) # Enter your PAT Key.
    username = "MikeMNelhams"
    main_branch_bool = input("Main Branch: Yes or No? ")
    yes_synonyms = ["yes", "y", "yh", "1", "true"]
    if main_branch_bool.lower() in yes_synonyms: 
        branch = "master" 
    else: 
        branch = "PROTOTYPE"

    user = g.get_user(username)
    r_proj_clone = 0
    for repo in g.get_user().get_repos():
        if repo.name == "ai-group-project-Team-JMJM":
            r_proj_clone = repo
            break
        # To see all the available attributes and methods
        print(dir(repo))
    if type(r_proj_clone) == int:
        print("ai-group-project-Team-JMJM not found")
        sys.exit()
    print("Importing Github cleaned text files...")
    contents = r_proj_clone.get_contents("RapLyrics/CLEAN", ref=branch)
    RAP_DATA = []
    for file_ in contents:
        path = file_.path
        path = str(path) 
        # Only choose the .txt files
        if path[-4:] == '.txt':
            # Append the Lyrics
            RAP_DATA.append(file_.decoded_content.decode("utf-8")) 
    
    temp_path = Path(path_name)
    if temp_path.is_file(): 
        if os.stat(path_name).st_size == 0:
            write_bool2 = True
        else: 
            write_bool2 = False
    else: 
        write_bool2 = True
    
    if write_bool2: 
        for lyric in RAP_DATA: 
            try:
                with open(path_name, 'w') as writefile: 
                    writefile.write(lyric)
            except: 
                print("Error, file moved/deleted during write")
        print("{} is now up to date!".format(path_name))
    else: 
        print("{} is already up to date!".format(path_name))

    contents = r_proj_clone.get_contents("RapLyrics/Other", ref=branch)
    for counter, file_ in enumerate(contents):
        path = file_.path
        path = str(path) 

        title_start = path.find('Other')
        title_len = path[title_start:].find('.')
        path = path[title_start + 6:title_start + title_len + 4]
        
        print("Writing file {} {}".format(counter, path))
        temp_path = Path(path)
        if temp_path.is_file():
            with open(path,'w'): pass # Cheeky way to clear the file if it exists
        
        # Split the long string into a list of lines, then split by words, then put into a csv, then to numpy array 
        data = file_.decoded_content
        data = data.decode('utf-8')[1:]

        with open(path, 'w') as writefile:
            writefile.write(data) 
    print("All files now up to date!")


def update_github(write_bool=False, path_name="AllLyrics.txt"):
    """
    Function for updating the github file, by cleaning the lyrics, optional write to txt file. 
    write_bool: bool
    path_name: str
    output: None
    """
    g = Github(getpass("Enter your PAT key ")) # Enter your PAT Key.
    username = "MikeMNelhams"
    main_branch_bool = input("Main Branch: Yes or No? ")
    yes_synonyms = ["yes", "y", "yh", "1", "true"]
    if main_branch_bool.lower() in yes_synonyms: 
        branch = "master" 
    else: 
        branch = "PROTOTYPE"

    user = g.get_user(username)
    r_proj_clone = 0
    for repo in g.get_user().get_repos():
        if repo.name == "ai-group-project-Team-JMJM":
            r_proj_clone = repo
            break
        # To see all the available attributes and methods
        print(dir(repo))
    
    if not r_proj_clone:
        print("ai-group-project-Team-JMJM not found")
        sys.exit()

    print("Importing editing csv files...")

    # Split the long string into a list of lines, then split by words, then put into a csv, then to numpy arr
    contents = r_proj_clone.get_contents("RapLyrics/Other", ref=branch)
    for counter, file_ in enumerate(contents):
        path = file_.path 
        path = str(path)
        title_start = path.find('Other')
        title_len = path[title_start:].find('.')
        name = path[title_start + 6:title_start + title_len + 4]
        print("Writing file {} {}".format(counter, name))
        if name.lower() == "censors.csv":
            censors = _decode_and_write(file_, name)
        elif name.lower() == "capitals.csv":
            capitals = _decode_and_write(file_, name)
        else: 
            _decode_and_write(file_, name)
    print("All editing csv files are up to date!")

    print("Importing Github uncleaned text files...")
    contents = r_proj_clone.get_contents("RapLyrics/UNCLEAN", ref=branch)

    RAP_DATA = []
    rap_lyric_names = []

    for file_ in contents:
        path = file_.path
        path = str(path) 
        # Only choose the .txt files
        if path[-4:] == '.txt':
            # Append the name
            title_start = path.find('UNCLEAN')
            title_len = path[title_start:].find('.')
            name = path[title_start + 8:title_start + title_len]
            if name[-2:] == 'UC':
                name = name[:-2]
            rap_lyric_names.append(name) 

        # Append the Lyrics
        RAP_DATA.append(file_.decoded_content.decode("utf-8")) 
        
    # Remove the \ufeff at the beginning O(n)
    for count, lyric in enumerate(RAP_DATA): 
        RAP_DATA[count] = lyric[1:]

    # Censor the profanities O(n*m + n*m2) m > m2 xor m2 > m
    for count in range(len(RAP_DATA)): 
        for i in range(len(censors[0:])):
            RAP_DATA[count] = RAP_DATA[count].replace(str(censors[i, 0]), str(censors[i, 1]))
        for i in range(len(capitals[0:])):
            RAP_DATA[count] = RAP_DATA[count].replace(str(capitals[i, 0]), str(capitals[i, 1]))

    contents = r_proj_clone.get_contents("RapLyrics/CLEAN", ref=branch)
    cleaned_names = []
    for counter, file_ in enumerate(contents):
        path = file_.path
        path = str(path) 
        print("File {} ".format(counter + 1) + path)
        # Only choose the .txt files
        if path[-4:] == '.txt':
            # Append the name
            title_start = path.find('CLEAN')
            title_len = path[title_start:].find('.')
        name = path[title_start + 6:title_start + title_len]
        if name[-2:] == 'CL':
            name = name[:-2]
        cleaned_names.append(name) 

    # ALL OF THE EDITING IS DONE IN THE 'PROTOTYPE BRANCH' to avoid overwriting import changes
    # If the (now cleaned) rap_lyrics name is new (not in cleaned_names), then we want to create that as a new file 
    # If the (now cleaned) rap_lyrics name is NOT new (not in cleaned_names), then we want to update the file
    # print(rap_lyric_names)
    # print(cleaned_names)
    print("Commiting files to github...")
    for counter, new_name in enumerate(rap_lyric_names): 
        if new_name in cleaned_names: 
            duplicate = r_proj_clone.get_contents("RapLyrics/CLEAN/{}CL.txt".format(new_name), ref=branch)
            r_proj_clone.update_file("RapLyrics/CLEAN/{}CL.txt".format(new_name), "This was uploaded automatically via pipeline", RAP_DATA[counter], duplicate.sha, branch=branch)
        else:
            r_proj_clone.create_file("RapLyrics/CLEAN/{}CL.txt".format(new_name), "This was uploaded automatically via pipeline", RAP_DATA[counter], branch=branch)

    if write_bool: 
        print("Writing text file to: {}".format(path_name))
        with open(path_name, 'w') as writefile:
            for lyric in RAP_DATA:
                writefile.write(lyric)


def generate_rap(Vocabulary):
    start_line = line_generator(Vocabulary)
    all_other_lines = [line_generator(Vocabulary) for i in range(999)]
    rap = [start_line]
    
    for i in range (19):
        next_line = next_line_stop_count(rap[len(rap) - 1], all_other_lines)
        all_other_lines.remove(next_line)
        rap.append(next_line)
    return rap


# Generate text
def line_generator(Vocabulary):
    index = 1
    chain = {}
    count = random.randint(6, 12) # Why 6 and 12?
    
    for word in Vocabulary[index:]:
        key = Vocabulary[index-1]
        if key in chain:
            chain[key].append(word)
        else:
            chain[key] = [word]
        index += 1
        
    word1 = random.choice(list(chain.keys()))
    line = word1.capitalize()

    while len(line.split(' ')) < count:
        word2 = random.choice(chain[word1])
        word1 = word2
        line += ' ' + word2.lower()
    return line


# Rhyme Functions
def reverse_syllable_extract(text):
    sy_form = []
    characters = [char for char in text]
    sylls = ['a', 'e', 'i', 'o', 'u']
    for x in characters:
        if x in sylls:
            sy_form.append(x)
    sy_form.reverse()
    return sy_form


def rev_syllable_stop_count(text1, text2):
    count = True 
    i = 0
    counter = 0
    syll1 = reverse_syllable_extract(text1)
    syll2 = reverse_syllable_extract(text2)
    while count == True:
        if i < min(len(syll1), len(syll2)) and syll1[i] == syll2[i]:
            counter += 1
            i += 1
        else:
            count = False
    return counter


def next_line_stop_count(start_line, lines):
    sy_lines = []
    for i in lines:
        sy_lines.append(rev_syllable_stop_count(start_line, i))
    choice = sy_lines[0]
    count = 0
    for i in range(len(sy_lines)):
        if sy_lines[i] > choice:
            choice = sy_lines[i]
    return lines[sy_lines.index(choice)]

In [ ]:
#@title Function Definitions RNN
def _split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

def train_RNN(sequence_length=100, batch_length=64, path_="AllLyrics.txt", rnn_units_ = 1024, epochs=10):
    # https://www.tensorflow.org/tutorials/text/text_generation
    # Step 1
    text = open(path_, 'rb').read().decode(encoding='utf-8')
    vocab = sorted(set(text))  # List of vocab
    chars = tf.strings.unicode_split(re.split('\s\n',text), input_encoding='UTF-8')  # Byte vector list object 
    ids_from_chars = preprocessing.StringLookup(vocabulary=list(vocab))  # List of ID object 

    # Step 2
    ids = ids_from_chars(chars)  # Convert from tokens + paddings
    chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True)  # Does something
    chars = chars_from_ids(ids)

    # Step 3
    all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
    ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

    def _text_from_ids(ids):
        return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

    # Test 1
    # for ids in ids_dataset.take(10):
    #    print(chars_from_ids(ids).numpy().decode('utf-8'))

    # Step 4
    seq_length = sequence_length
    examples_per_epoch = len(text)//(seq_length+1)
    sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

    # Test 2 
    # for seq in sequences.take(1):
    #    print(chars_from_ids(seq))
    # Test 3 - Back from tokens 
    # for seq in sequences.take(5):
    #    print(_text_from_ids(seq).numpy())

    # Step 5 - Create training examples and targets
    dataset = sequences.map(_split_input_target)
    # Test 3
    # for input_example, target_example in  dataset.take(1):
    #    print("Input :", _text_from_ids(input_example).numpy())
    #    print("Target:", _text_from_ids(target_example).numpy())
    
    # Step 6 - Create training batches 
    # Batch size
    BATCH_SIZE = batch_length

    # Buffer size to shuffle the dataset
    # (TF data is designed to work with possibly infinite sequences,
    # so it doesn't attempt to shuffle the entire sequence in memory. Instead,
    # it maintains a buffer in which it shuffles elements).
    BUFFER_SIZE = 10000

    dataset = (
        dataset
        .shuffle(BUFFER_SIZE)
        .batch(BATCH_SIZE, drop_remainder=True)
        .prefetch(tf.data.experimental.AUTOTUNE))
    
    # Step 7 - Build the model
    # Length of the vocabulary in chars
    vocab_size = len(vocab)

    # The embedding dimension
    embedding_dim = 256

    # Number of RNN units
    rnn_units = rnn_units_

    class MyModel(tf.keras.Model):
        def __init__(self, vocab_size, embedding_dim, rnn_units):
            super().__init__(self)
            self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
            self.gru = tf.keras.layers.GRU(rnn_units,
                                        return_sequences=True, 
                                        return_state=True)
            self.dense = tf.keras.layers.Dense(vocab_size)

        def call(self, inputs, states=None, return_state=False, training=False):
            x = inputs
            x = self.embedding(x, training=training)
            if states is None:
                states = self.gru.get_initial_state(x)
            x, states = self.gru(x, initial_state=states, training=training)
            x = self.dense(x, training=training)

            if return_state:
                return x, states
            else: 
                return x

    model = MyModel(
        # Be sure the vocabulary size matches the `StringLookup` layers.
        vocab_size=len(ids_from_chars.get_vocabulary()),
        embedding_dim=embedding_dim,
        rnn_units=rnn_units)
    
    for input_example_batch, target_example_batch in dataset.take(1):
        example_batch_predictions = model(input_example_batch)
        print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")
    
    model.summary()

    # Step 8 - Train the model
    loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
    example_batch_loss = loss(target_example_batch, example_batch_predictions)
    mean_loss = example_batch_loss.numpy().mean()
    print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
    print("Mean loss:        ", mean_loss)

    model.compile(optimizer='adam', loss=loss)

    # Directory where the checkpoints will be saved
    checkpoint_dir = './training_checkpoints'
    # Name of the checkpoint files
    checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_prefix,
        save_weights_only=True)
    
    EPOCHS = epochs  # Number of training steps 
    history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

    class OneStep(tf.keras.Model):
        def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
            super().__init__()
            self.temperature=temperature
            self.model = model
            self.chars_from_ids = chars_from_ids
            self.ids_from_chars = ids_from_chars

            # Create a mask to prevent "" or "[UNK]" from being generated.
            skip_ids = self.ids_from_chars(['','[UNK]'])[:, None]
            sparse_mask = tf.SparseTensor(
                # Put a -inf at each bad index.
                values=[-float('inf')]*len(skip_ids),
                indices = skip_ids,
                # Match the shape to the vocabulary
                dense_shape=[len(ids_from_chars.get_vocabulary())]) 
            self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    
        @tf.function
        def generate_one_step(self, inputs, states=None):
            # Convert strings to token IDs.
            input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
            input_ids = self.ids_from_chars(input_chars).to_tensor()

            # Run the model.
            # predicted_logits.shape is [batch, char, next_char_logits] 
            predicted_logits, states =  self.model(inputs=input_ids, states=states, 
                                                return_state=True)
            # Only use the last prediction.
            predicted_logits = predicted_logits[:, -1, :]
            predicted_logits = predicted_logits/self.temperature
            # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
            predicted_logits = predicted_logits + self.prediction_mask

            # Sample the output logits to generate token IDs.
            predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
            predicted_ids = tf.squeeze(predicted_ids, axis=-1)

            # Convert from token ids to characters
            predicted_chars = self.chars_from_ids(predicted_ids)

            # Return the characters and model state.
            return predicted_chars, states
    
    one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

    start = time.time()
    states = None
    next_char = tf.constant([' ', ' ', ' ', ' ', ' '])
    result = [next_char]

    for n in range(1000):
        next_char, states = one_step_model.generate_one_step(next_char, states=states)
        result.append(next_char)

    result = tf.strings.join(result)
    end = time.time()

    print(result, '\n\n' + '_'*80)
    print(f"\nRun time: {end - start}")

    tf.saved_model.save(one_step_model, 'one_step')

    print(tf.strings.join(result).numpy().decode("utf-8"))


def generate_rap_RNN(line_start='It', num_chars=100):
    # Currently does not work, since google colab is very finicky with saving big files...
    one_step_reloaded = tf.saved_model.load('one_step')
    states = None
    next_char = tf.constant([line_start])
    result = [next_char]

    for _ in range(num_chars):
        next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
        result.append(next_char)

    print(tf.strings.join(result).numpy().decode("utf-8"))

In [ ]:
#@title Example model
train_RNN(epochs=50)
#generate_rap_RNN()

(64, 100, 109) # (batch_size, sequence_length, vocab_size)
Model: "my_model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     multiple                  27904     
_________________________________________________________________
gru_12 (GRU)                 multiple                  3938304   
_________________________________________________________________
dense_12 (Dense)             multiple                  111725    
Total params: 4,077,933
Trainable params: 4,077,933
Non-trainable params: 0
_________________________________________________________________
Prediction shape:  (64, 100, 109)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.6926003
Epoch 1/50
211/211 [==============================] - 13s 53ms/step - loss: 3.2603
Epoch 2/50
211/211 [==============================] - 12s 53ms/step - loss: 2.0553
Epoch 3/50
211/211 [======================

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


 it's elloyour fate
In a B Fly shut
Is this pic, long eight switch b*****
Wanta suck my d*ck in the trailer valuman
Trynn's to watch my hustle with surprising, I ain't love me when I wait, we're in the Bee'?
Bleed stop right we look released
But yet that's why my breathed and attrestep with me like K either New York Knine He the NBB
These creats suckers
Ty life these b*****es
Yeah yeah
Jung Doneven
Yall eat piranin and the album though My flashy sy
Cant let it go it aint my fault
I wouldnt rap like a bum of your kids good
But I can rending but I just been busy
The sunsetter doors
Yeah he in the vanson people better gree enough
This very money I make you white bemome bring ball ninjas
What you wanna see
Baby the type to be fricked with
They gone tome back this the way you lie
I got game spitta end
Shouth hair to about honey Jam
And it was naked Lavor Keep em regal
Im maybe what she a standin to men
Thank a death weeke club up when keep ya didnt firs What
You know I scrutch for sex is li

In [ ]:
# Import all of Mike's lyrics. PATKEY: 5ae2446bd5828c9e27deb3865118d9e783aa6e15
import_github()

Enter your PAT key ··········
Main Branch: Yes or No? Yes
Importing editing csv files...
Writing file 0 capitals.csv
Writing file 1 censors.csv
All editing csv files are up to date!
Importing Github uncleaned text files...
File 1 RapLyrics/CLEAN/Kanye_WestCL.txt
File 2 RapLyrics/CLEAN/Lil_WayneCL.txt
File 3 RapLyrics/CLEAN/eminemCL.txt
File 4 RapLyrics/CLEAN/ludacrisCL.txt
File 5 RapLyrics/CLEAN/nicki-minajCL.txt
File 6 RapLyrics/CLEAN/notorious-bigCL.txt
Commiting files to github...
Writing text file to: AllLyrics.txt


In [ ]:
# Extract all of Mike's lyrics. (Markov Model)
Text = open("AllLyrics.txt", "r").read()
Vocabulary = ''.join([i for i in Text if not i.isdigit()]).replace("\n", " ").split(' ')
# print(Vocabulary)
# line_generator(Vocabulary)

In [ ]:
for line in generate_rap(Vocabulary):
    print(line)

Violate me thats my decimals damn ma i reach
Theyll laugh when i dare you can hook b that drive da
Ills holdin it doesnt ran up what i mean
Making money and it rub it doesnt it say i mean
Peel more money much you love ma little nasty
Lap chronic by tomorrow all i sing real
Brains blown out my bag steamin girls steady
Prey hey dad place when i b smoking mics like a
Creepin two rides than biggie biggie smalls
Lavender and on toes b***** in jay getting high reach
Friend rog whats your clip hit me as
Thou shalt not guilty richer than
Happennin out here for the winter far
Rosewood and wednesdays while we so why cause im dead
Cop all day another struggle i yeah
May be classified in the stash
Comer then i rep smokin sacks up the funk ill be what
Camera rollin blunts left but my nine at
Dukes a place why it on sky is heat
Bacon backspins to make the stars


In [ ]:
# Now move to generate a 20 line rap
# Will generate 1000 lines using line_generator then from first generated line pick next best line by rhyme 19 times
# I've left it as a list with strings as the lines, we can have fun improving things if we think this is a good start
# The above has taken ages to load as has made 1000 lines then compared each one to try and find the best next!
# Also the vocab is much longer now